In [1]:
import pandas as pd
import numpy as np
import pymssql

In [2]:
mssql_conn = pymssql.connect(server='epdm', user='sa', password='Pdmepdme1!',database='ePDM_Elcam')

In [29]:
#test connection
query='SELECT Filename, LockPath, LockDomain, LockDate, Username, Email from [ePDM_Elcam].[dbo].[Documents] INNER JOIN [ePDM_Elcam].[dbo].[Users] on [ePDM_Elcam].[dbo].[Documents].UserID = [ePDM_Elcam].[dbo].[Users].UserID WHERE LockDate IS NOT NULL AND CurrentStatusid IS NOT NULL AND Deleted=0 ORDER BY Username ASC'
#run query
df = pd.read_sql(query, mssql_conn)


c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [36]:
res ={}
for i in df['Username'].unique():
    res[i] = df[df['Username']==i]['Filename'].count()
res=pd.DataFrame.from_dict(res, orient='index')
res.sort_values(   by=0, ascending=False, inplace=True)
res


,0
hen.b,171
galei,147
sergei.h,85
eyal.sh,50
hillel.y,39
...,...
daphna.p,1
ben.e,1
AVI,1
alon.z,1


In [46]:
df2=pd.read_csv("kimaat.moshlam.csv")
#rename the first column to Username
df2
# for i in range(len(df2)):
#     row=df2.iloc[i]
#     if row.EmailAddress=='' and row.ManagerEmail=='':
#         print(row.Username)

,DisplayName,EmailAddress,ManagerEmail
0,Administrator,Administrator@elcam.co.il,NaN
1,tsachi shaked,tsachi@elcam.co.il,NaN
2,amirb,AMIRB@elcam.co.il,guli@elcam.co.il
3,itai.galed,itai@elcam.co.il,ran@elcam.co.il
4,cognosAdmin,NaN,NaN
...,...,...,...
1274,Sirin Khir,NaN,Shadia@elcam.co.il
1275,Saleem Break,saleem.b@elcam.co.il,natasha.g@elcam.co.il
1276,Francesco Riccardo Della Casa,francesco.dellacasa@elcam-medical.com,NaN
1277,Michael Miller,NaN,ozi@elcam.co.il
